In [2]:
!pip install -Uq sagemaker
!pip install  "datasets[s3]==2.16" 
!pip install torch
!pip install transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.33.13 requires botocore==1.34.131, but you have botocore 1.35.65 which is incompatible.
  Using cached datasets-2.16.0-py3-none-any.whl.metadata (20 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached fsspec-2023.10.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
  Using cached aiohappyeyeballs-2.4.3-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.5.0-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Us

In [8]:
import boto3
import sagemaker
import sagemaker.session
from sagemaker.workflow.pipeline_context import PipelineSession

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()

pipeline_session = PipelineSession()

model_package_group_name = f"AbaloneModelPackageGroupName"



# !mkdir -p data
# local_path = "data/abalone-dataset.csv"

# s3 = boto3.resource("s3")
# s3.Bucket(f"sagemaker-servicecatalog-seedcode-{region}").download_file(
#     "dataset/abalone-dataset.csv",
#     local_path
# )

# base_uri = f"s3://{default_bucket}/abalone"
# input_data_uri = sagemaker.s3.S3Uploader.upload(
#     local_path=local_path, 
#     desired_s3_uri=base_uri,
# )
# print(input_data_uri)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
# local_path = "data/abalone-dataset-batch.csv"

# s3 = boto3.resource("s3")
# s3.Bucket(f"sagemaker-servicecatalog-seedcode-{region}").download_file(
#     "dataset/abalone-dataset-batch",
#     local_path
# )

# base_uri = f"s3://{default_bucket}/abalone"
# batch_data_uri = sagemaker.s3.S3Uploader.upload(
#     local_path=local_path, 
#     desired_s3_uri=base_uri,
# )
# print(batch_data_uri)

In [5]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
model_approval_status = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval"
)
# input_data = ParameterString(
#     name="InputData",
#     default_value=input_data_uri,
# )
# batch_data = ParameterString(
#     name="BatchData",
#     default_value=batch_data_uri,
# )

In [9]:
!mkdir -p abalone
!mkdir -p scripts

In [7]:
# %%writefile abalone/preprocessing.py


# import os
# import pandas as pd

# # Define input and output paths based on your ProcessingInput and ProcessingOutput settings
# input_path = "/opt/ml/processing/input"
# train_output_path = "/opt/ml/processing/train/train.csv"
# validation_output_path = "/opt/ml/processing/validation/validation.csv"
# test_output_path = "/opt/ml/processing/test/test.csv"

# def main():
#     print("######################### PROCESSING STARTED ... ################################")
    
#     # Step 1: Dummy reading of the input file
#     # Check if the input file exists and simulate a dummy DataFrame
#     if os.path.exists(f"{input_path}/shout.jpg"):
#         print("Input file found. Creating dummy data.")

#     # Step 2: Create a dummy DataFrame
#     data = {
#         "feature1": [1, 2, 3, 4, 5],
#         "feature2": [5, 4, 3, 2, 1],
#         "label": [0, 1, 0, 1, 0]
#     }
#     df = pd.DataFrame(data)

#     # Step 3: Write dummy DataFrame to train, validation, and test output paths
#     df.to_csv(train_output_path, index=False)
#     df.to_csv(validation_output_path, index=False)
#     df.to_csv(test_output_path, index=False)

#     print("Dummy preprocessing completed. Output files generated.")

# if __name__ == "__main__":
#     main()

# print("######################### PROCESSING ENDED ... ################################")

In [10]:
%%writefile abalone/preprocessing.py

import os
import shutil

def copy_files(input_dir, output_dir):
    """
    Copies all files and subdirectories from input_dir to output_dir.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for item in os.listdir(input_dir):
        src_path = os.path.join(input_dir, item)
        dest_path = os.path.join(output_dir, item)
        if os.path.isdir(src_path):
            shutil.copytree(src_path, dest_path, dirs_exist_ok=True)
            print(f"Copied directory {src_path} to {dest_path}")
        else:
            shutil.copy2(src_path, dest_path)
            print(f"Copied file {src_path} to {dest_path}")

if __name__ == "__main__":
    base_dir = "/opt/ml/processing/input/train"
    base_dir2 = "/opt/ml/processing/input/test"
    train_output_dir = "/opt/ml/processing/output/train"
    test_output_dir = "/opt/ml/processing/output/test"

    # Copy files for train
    copy_files(base_dir, train_output_dir)

    # Copy files for test
    copy_files(base_dir2, test_output_dir)

    print("File copying completed.")


    print("######################### PROCESSING ENDED ... ################################")

Overwriting abalone/preprocessing.py


In [11]:
%%writefile ./scripts/train.py

from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import load_from_disk
import random
import logging
import sys
import argparse
import os
import torch

if __name__ == "__main__":

    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--epochs", type=int, default=3)
    parser.add_argument("--train_batch_size", type=int, default=32)
    parser.add_argument("--eval_batch_size", type=int, default=64)
    parser.add_argument("--warmup_steps", type=int, default=500)
    parser.add_argument("--model_name", type=str)
    parser.add_argument("--learning_rate", type=str, default=5e-5)

    # Data, model, and output directories
    parser.add_argument("--output_data_dir", type=str, default=os.environ["SM_OUTPUT_DATA_DIR"])
    parser.add_argument("--model_dir", type=str, default=os.environ["SM_MODEL_DIR"])
    parser.add_argument("--n_gpus", type=str, default=os.environ["SM_NUM_GPUS"])
    parser.add_argument("--training_dir", type=str, default=os.environ["SM_CHANNEL_TRAIN"])
    parser.add_argument("--test_dir", type=str, default=os.environ["SM_CHANNEL_TEST"])

    args, _ = parser.parse_known_args()

    # Set up logging
    logger = logging.getLogger(__name__)

    logging.basicConfig(
        level=logging.getLevelName("INFO"),
        handlers=[logging.StreamHandler(sys.stdout)],
        format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    )

    # load datasets
    train_dataset = load_from_disk(args.training_dir)
    test_dataset = load_from_disk(args.test_dir)

    logger.info(f" loaded train_dataset length is: {len(train_dataset)}")
    logger.info(f" loaded test_dataset length is: {len(test_dataset)}")

    # compute metrics function for binary classification
    def compute_metrics(pred):
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)
        precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
        acc = accuracy_score(labels, preds)
        return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

    # download model from model hub
    model = AutoModelForSequenceClassification.from_pretrained(args.model_name)
    tokenizer = AutoTokenizer.from_pretrained(args.model_name)

    # define training args
    training_args = TrainingArguments(
        output_dir=args.model_dir,
        num_train_epochs=args.epochs,
        per_device_train_batch_size=args.train_batch_size,
        per_device_eval_batch_size=args.eval_batch_size,
        warmup_steps=args.warmup_steps,
        evaluation_strategy="epoch",
        logging_dir=f"{args.output_data_dir}/logs",
        learning_rate=float(args.learning_rate),
    )

    # create Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
    )
    # train model
    trainer.train()

    # evaluate model
    eval_result = trainer.evaluate(eval_dataset=test_dataset)

    # writes eval result to file which can be accessed later in s3 ouput
    with open(os.path.join(args.output_data_dir, "eval_results.txt"), "w") as writer:
        print(f"***** Eval results *****")
        for key, value in sorted(eval_result.items()):
            writer.write(f"{key} = {value}\n")

    # Saves the model to s3
    trainer.save_model(args.model_dir)

Overwriting ./scripts/train.py


In [12]:
from datasets import load_dataset
from transformers import AutoTokenizer

# tokenizer used in preprocessing
tokenizer_name = 'distilbert-base-uncased'

# dataset used
dataset_name = 'imdb'

# s3 key prefix for the data
s3_prefix = 'samples/datasets/imdb'

# load dataset
# dataset = load_dataset(dataset_name)

# download tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# tokenizer helper function
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

# load dataset
train_dataset, test_dataset = load_dataset('imdb', split=['train', 'test'])
test_dataset = test_dataset.shuffle().select(range(10000)) # smaller the size for test dataset to 10k 


# tokenize dataset
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# set format for pytorch
train_dataset =  train_dataset.rename_column("label", "labels")
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset = test_dataset.rename_column("label", "labels")
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

# save train_dataset to s3
training_input_path = f's3://{sagemaker_session.default_bucket()}/{s3_prefix}/train'
train_dataset.save_to_disk(training_input_path)

# save test_dataset to s3
test_input_path = f's3://{sagemaker_session.default_bucket()}/{s3_prefix}/test'
test_dataset.save_to_disk(test_input_path)

[11/20/24 20:51:42] INFO     PyTorch version 2.5.1 available.                                          ]8;id=215804;file:///opt/conda/lib/python3.11/site-packages/datasets/config.py\config.py]8;;\:]8;id=497902;file:///opt/conda/lib/python3.11/site-packages/datasets/config.py#58\58]8;;\

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/opt/conda/lib/python3.11/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


Saving the dataset (0/1 shards):   0%|          | 0/25000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [13]:
from sagemaker.sklearn.processing import SKLearnProcessor


framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type="ml.m5.xlarge",
    instance_count=processing_instance_count,
    base_job_name="sklearn-abalone-process",
    sagemaker_session=pipeline_session,
    role=role,
)

[11/20/24 20:52:12] INFO     Defaulting to only available Python version: py3                     ]8;id=751122;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=238920;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#603\603]8;;\

In [14]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
   

processor_args = sklearn_processor.run(
    inputs=[
      ProcessingInput(source=f's3://{sagemaker_session.default_bucket()}/{s3_prefix}/train', destination="/opt/ml/processing/input/train"),
      ProcessingInput(source=f's3://{sagemaker_session.default_bucket()}/{s3_prefix}/test', destination="/opt/ml/processing/input/test"),  
        
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/output/train"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/output/test")
    ],
    code="abalone/preprocessing.py",
) 

step_process = ProcessingStep(
    name="AbaloneProcess",
    step_args=processor_args
)

/opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [15]:
model_path = f"s3://{default_bucket}/AbaloneTrain"

In [16]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 1,
                 'train_batch_size': 16,
                 'eval_batch_size' : 32,
                 'model_id':'distilbert-base-uncased'
                }

In [17]:
from sagemaker.workflow.pipeline_context import PipelineSession
pipeline_session = PipelineSession()

huggingface_estimator = HuggingFace(entry_point='train.py',
                            source_dir='./scripts',
                            base_job_name='huggingface-sdk-extension',
                            instance_type='ml.p3.2xlarge',
                            instance_count=1,
                            transformers_version="4.28.1",                             # Transformers version used
                            pytorch_version="2.0.0",                                  # PyTorch version used
                            py_version='py310', 
                            role=role,
                            sagemaker_session  = pipeline_session,
                            hyperparameters = {'epochs': 1,
                                               'train_batch_size': 32,
                                               'model_name':'distilbert-base-uncased'
                                                })

In [18]:
# import inspect
# x = inspect.getsource(huggingface_estimator.fit)
# print(x)

In [19]:
# from sagemaker.huggingface.model import HuggingFaceModel

# hub = {
#   'HF_MODEL_ID':'vikhyatk/moondream2', # model_id from hf.co/models
#   'HF_TASK':'image-to-text'                           # NLP task you want to use for predictions
# }

# # create Hugging Face Model Class
# huggingface_model = HuggingFaceModel(
#    env=hub,                                                # configuration for loading model from Hub
#    role=role,                                              # IAM role with permissions to create an endpoint
#    transformers_version="4.28.1",                             # Transformers version used
#    pytorch_version="2.0.0",                                  # PyTorch version used
#    py_version='py310',                                      # Python version used
# )

In [20]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

from transformers import TrainingArguments
# train_args = TrainingArguments("working_dir")
train_args = huggingface_estimator.fit({'train': TrainingInput(
    s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
), 'test': TrainingInput(
    s3_data=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
)})

step_train = TrainingStep(
    name="AbaloneTrain",
    step_args = train_args
)

In [21]:
from sagemaker.model import Model
from sagemaker.workflow.model_step import ModelStep

model = Model(
    image_uri=huggingface_estimator.training_image_uri(),
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=pipeline_session,
    role=role,
)

step_model_create = ModelStep(
   name="MyModelCreationStep",
   step_args=model.create(instance_type="ml.m5.xlarge"),
)

[11/20/24 20:52:21] INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=706258;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=369341;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#674\674]8;;\
                             instance_type, framework etc.                                                         

In [83]:
# from sagemaker.model import Model
# from sagemaker.workflow.model_step import ModelStep

# model = Model(
#     image_uri=huggingface_estimator.training_image_uri(),
#     model_data="s3://sagemaker-eu-central-1-643202173500/pipelines-8f0u2hnc4dwk-AbaloneTrain-tjTIFxARlg/output/model.tar.gz",
#     sagemaker_session=pipeline_session,
#     role=role,
# )

# step_model_create = ModelStep(
#    name="MyModelCreationStep",
#    step_args=model.create(instance_type="ml.m5.xlarge"),
# )



[11/20/24 23:03:18] INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=550895;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=378858;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#674\674]8;;\
                             instance_type, framework etc.                                                         

In [23]:
# step_model_register = ModelStep(
#    name="MyModelRegisterStep",
#    step_args=model.register(),
# )

In [28]:
from sagemaker import PipelineModel

pipeline_model = PipelineModel(
   models=[model],
   role=role,sagemaker_session=pipeline_session,
)

register_model_step_args = pipeline_model.register(
    content_types=["application/json"],
   response_types=["application/json"],
   inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
   transform_instances=["ml.m5.xlarge"],
   model_package_group_name='sipgroup',
)

step_model_registration = ModelStep(
   name="AbaloneRegisterModel",
   step_args=register_model_step_args,
)

In [3]:
import boto3
from datetime import datetime

def get_latest_approved_model():
    # Initialize SageMaker client
    sm_client = boto3.client('sagemaker', region_name='eu-central-1')  # Change region as needed

    # List all model packages
    response = sm_client.list_model_packages(
        ModelApprovalStatus="Approved",  # Fetch only models with "Approved" status
        SortBy="CreationTime",           # Sort by creation time
        SortOrder="Descending"           # Get the latest models first
    )
    print("response is ", response)

    # Extract the latest approved model
    if 'ModelPackageSummaryList' in response and response['ModelPackageSummaryList']:
        latest_model = response['ModelPackageSummaryList'][0]
        return {
            "ModelPackageArn": latest_model["ModelPackageArn"],
            "CreationTime": latest_model["CreationTime"],
            "ModelApprovalStatus": latest_model["ModelApprovalStatus"],
        }
    else:
        return "No approved models found."

latest_model = get_latest_approved_model()
print(f"Latest Approved Model: {latest_model}")


response is  {'ModelPackageSummaryList': [], 'ResponseMetadata': {'RequestId': '4b671111-a6f7-49cb-ac6c-0f5298da84b0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4b671111-a6f7-49cb-ac6c-0f5298da84b0', 'content-type': 'application/x-amz-json-1.1', 'content-length': '30', 'date': 'Fri, 22 Nov 2024 10:20:23 GMT'}, 'RetryAttempts': 0}}
Latest Approved Model: No approved models found.


In [6]:
import boto3

def get_latest_approved_model():
    # Initialize SageMaker client
    sm_client = boto3.client('sagemaker', region_name='eu-central-1')  # Change region as needed

    # Step 1: List all model package groups
    response = sm_client.list_model_package_groups(SortBy="CreationTime", SortOrder="Descending")
    groups = response.get("ModelPackageGroupSummaryList", [])
    
    if not groups:
        return "No model package groups found."

    latest_model = None

    # Step 2: Iterate through each group to find approved models
    for group in groups:
        group_name = group["ModelPackageGroupName"]
        print(f"Checking group: {group_name}")
        
        # Fetch packages for the group
        packages = sm_client.list_model_packages(
            ModelPackageGroupName=group_name,
            ModelApprovalStatus="Approved",
            SortBy="CreationTime",
            SortOrder="Descending"
        )
        

        # If packages exist, get the latest one
        if packages.get("ModelPackageSummaryList"):
            latest_in_group = packages["ModelPackageSummaryList"][0]  # Most recent in this group

            # Compare to find the latest overall
            if not latest_model or latest_in_group["CreationTime"] > latest_model["CreationTime"]:
                latest_model = latest_in_group

    # Step 3: Return the latest approved model
    if latest_model:
        return {
            "ModelPackageArn": latest_model["ModelPackageArn"],
            "CreationTime": latest_model["CreationTime"],
            "ModelApprovalStatus": latest_model["ModelApprovalStatus"],
            "ModelPackageGroupName": latest_model.get("ModelPackageGroupName"),
        }
    else:
        return "No approved models found in any group."

latest_model = get_latest_approved_model()
print(f"Latest Approved Model: {latest_model}")


Checking group: sipgroup
Checking group: huggingface-pytorch-training
Checking group: AbaloneModelPackageGroupName
Checking group: mlops-model-pkg
Checking group: test-model-package-group
Latest Approved Model: {'ModelPackageArn': 'arn:aws:sagemaker:eu-central-1:643202173500:model-package/huggingface-pytorch-training/1', 'CreationTime': datetime.datetime(2024, 11, 20, 21, 11, 27, 975000, tzinfo=tzlocal()), 'ModelApprovalStatus': 'Approved', 'ModelPackageGroupName': 'huggingface-pytorch-training'}


In [9]:
from sagemaker import ModelPackage

latest_model_arn = latest_model["ModelPackageArn"]


if isinstance(latest_model_arn, str) and latest_model_arn.startswith("arn:aws:sagemaker"):
    print(f"Latest Approved Model ARN: {latest_model_arn}")

    model = ModelPackage(
        role=role,
        model_package_arn=latest_model_arn,
        sagemaker_session=sagemaker_session
    )

    print(f"ModelPackage created with ARN: {latest_model_arn}")
else:
    print(latest_model_arn)

Latest Approved Model ARN: arn:aws:sagemaker:eu-central-1:643202173500:model-package/huggingface-pytorch-training/1
ModelPackage created with ARN: arn:aws:sagemaker:eu-central-1:643202173500:model-package/huggingface-pytorch-training/1


In [11]:
model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name = "mlops-cop-endpoint")

---------------------------------------------------------------------*

UnexpectedStatusException: Error hosting endpoint mlops-cop-endpoint: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html

In [29]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = f"AbalonePipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        model_approval_status
    ],
    steps=[step_process, step_train, step_model_create, step_model_registration],
)

In [30]:
import json

json.loads(pipeline.definition())

[11/20/24 21:00:30] WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=458663;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=436576;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=775119;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=408122;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#674\674]8;;\
                             instance_type, framework etc.                                                         

                    WARNING  Popping out 'TrainingJobName' from the pipeline definition by default ]8;id=589144;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=52096;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             since it will be overridden at pipeline execution time. Please                        
                             utilize the PipelineDefinitionConfig to persist this field in the                     
                             pipeline definition if desired.                                                       

                    WARNING  Popping out 'ModelName' from the pipeline definition by default since ]8;id=39970;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=594868;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             it will be overridden at pipeline execution time. Please utilize the                  
                             PipelineDefinitionConfig to persist this field in the pipeline                        
                             definition if desired.                                                                

                    WARNING  Popping out 'CertifyForMarketplace' from the pipeline definition since   ]8;id=442030;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/_utils.py\_utils.py]8;;\:]8;id=589770;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/_utils.py#515\515]8;;\
                             it will be overridden in pipeline execution time.                                     

                    WARNING  Popping out 'ModelPackageName' from the pipeline definition by        ]8;id=981466;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=738784;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceCount',
   'Type': 'Integer',
   'DefaultValue': 1},
  {'Name': 'ModelApprovalStatus',
   'Type': 'String',
   'DefaultValue': 'PendingManualApproval'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'AbaloneProcess',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': 'ml.m5.xlarge',
      'InstanceCount': {'Get': 'Parameters.ProcessingInstanceCount'},
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '492215442770.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3',
     'ContainerEntrypoint': ['python3',
      '/opt/ml/processing/input/code/preprocessing.py']},
    'RoleArn': 'arn:aws:iam::643202173500:role/SagemakerDomainRole',
    'ProcessingInputs': [{'InputName': 'input-1',
      'AppManaged': F

In [31]:
pipeline.upsert(role_arn=role)

[11/20/24 21:00:35] WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=625173;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=685697;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=428149;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=664690;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#674\674]8;;\
                             instance_type, framework etc.                                                         

                    WARNING  Popping out 'TrainingJobName' from the pipeline definition by default ]8;id=507666;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=306589;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             since it will be overridden at pipeline execution time. Please                        
                             utilize the PipelineDefinitionConfig to persist this field in the                     
                             pipeline definition if desired.                                                       

                    WARNING  Popping out 'ModelName' from the pipeline definition by default since ]8;id=826219;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=815659;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             it will be overridden at pipeline execution time. Please utilize the                  
                             PipelineDefinitionConfig to persist this field in the pipeline                        
                             definition if desired.                                                                

                    WARNING  Popping out 'ModelPackageName' from the pipeline definition by        ]8;id=841087;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=219694;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

...

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=695528;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=431596;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#674\674]8;;\
                             instance_type, framework etc.                                                         

                    WARNING  Popping out 'TrainingJobName' from the pipeline definition by default ]8;id=735530;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=616295;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             since it will be overridden at pipeline execution time. Please                        
                             utilize the PipelineDefinitionConfig to persist this field in the                     
                             pipeline definition if desired.                                                       

                    WARNING  Popping out 'ModelName' from the pipeline definition by default since ]8;id=520366;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=733541;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             it will be overridden at pipeline execution time. Please utilize the                  
                             PipelineDefinitionConfig to persist this field in the pipeline                        
                             definition if desired.                                                                

                    WARNING  Popping out 'ModelPackageName' from the pipeline definition by        ]8;id=782341;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=560017;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

{'PipelineArn': 'arn:aws:sagemaker:eu-central-1:643202173500:pipeline/AbalonePipeline',
 'ResponseMetadata': {'RequestId': 'e70b5eb3-e0d8-4638-8fc2-4237af0fe243',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e70b5eb3-e0d8-4638-8fc2-4237af0fe243',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '86',
   'date': 'Wed, 20 Nov 2024 21:00:35 GMT'},
  'RetryAttempts': 0}}

In [32]:
execution = pipeline.start()

In [33]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:eu-central-1:643202173500:pipeline/AbalonePipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:eu-central-1:643202173500:pipeline/AbalonePipeline/execution/csg84w2vunj8',
 'PipelineExecutionDisplayName': 'execution-1732136440525',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'abalonepipeline',
  'TrialName': 'csg84w2vunj8'},
 'CreationTime': datetime.datetime(2024, 11, 20, 21, 0, 40, 459000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 11, 20, 21, 0, 40, 459000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:eu-central-1:643202173500:user-profile/d-nosrgzkounei/raza',
  'UserProfileName': 'raza',
  'DomainId': 'd-nosrgzkounei',
  'IamIdentity': {'Arn': 'arn:aws:sts::643202173500:assumed-role/SagemakerDomainRole/SageMaker',
   'PrincipalId': 'AROAZLQOWJI6AQPICIFTX:SageMaker'}},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:eu-central-1:643202173500:user-

In [41]:
execution.list_steps()

[{'StepName': 'AbaloneProcess',
  'StartTime': datetime.datetime(2024, 11, 15, 14, 51, 12, 403000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 11, 15, 14, 51, 14, 184000, tzinfo=tzlocal()),
  'StepStatus': 'Failed',
  'FailureReason': 'ClientError: Failed to invoke sagemaker:CreateProcessingJob. Error Details: null',
  'Metadata': {},
  'AttemptCount': 1}]

In [42]:
role

'arn:aws:iam::643202173500:role/SagemakerDomainRole'

In [56]:
from sagemaker.enums import EndpointType
from sagemaker.compute_resource_requirements.resource_requirements import ResourceRequirements

resources = ResourceRequirements(
    requests = {
        "num_cpus": 2,  # Number of CPU cores required:
        "memory": 8192,  # Minimum memory required in Mb (required)
        "copies": 1,
    },
    limits = {},
)

predictor = model.deploy(
    initial_instance_count = 1,
    instance_type = "ml.m5.4xlarge", 
    endpoint_type = EndpointType.INFERENCE_COMPONENT_BASED,
    resources = resources,
)

[11/20/24 22:35:41] INFO     Creating endpoint-config with name                                     ]8;id=309539;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=536382;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#5820\5820]8;;\
                             huggingface-pytorch-inference-2024-11-2-2024-11-20-22-35-41-385                       

                    INFO     Creating endpoint with name                                            ]8;id=836762;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=294753;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4642\4642]8;;\
                             huggingface-pytorch-inference-2024-11-2-2024-11-20-22-35-41-385                       

----!

[11/20/24 22:38:12] INFO     Creating model with name:                                              ]8;id=481525;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=632666;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4025\4025]8;;\
                             huggingface-pytorch-inference-2024-11-20-22-38-12-547                                 

[11/20/24 22:38:13] INFO     Creating inference component with name                                 ]8;id=161743;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=750630;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4829\4829]8;;\
                             huggingface-pytorch-inference-2024-11-20-22-30--1732141851-edac for                   
                             endpoint                                                                              
                             huggingface-pytorch-inference-2024-11-2-2024-11-20-22-35-41-385                       

----------------!

In [58]:
res = predictor.predict({
    'inputs': 'This is a test sentence for BERT model prediction.'
})

print(res)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 res = predictor.predict({                                                                    │
│   2 │   'inputs': 'This is a test sentence for BERT model prediction.'                           │
│   3 })                                                                                           │
│   4                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/base_predictor.py:212 in predict               │
│                                                                                                  │
│   209 │   │   if inference_component_name:                                                       │
│   210 │   │   │   request_args["InferenceComponentName"] = inference_component_name              │
│   211 │   │                                                                                      │
│ ❱ 212 │   │   response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**req   │
│   213 │   │   return self._handle_response(response)                                             │
│   214 │                                                                                          │
│   215 │   def _handle_response(self, response):                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/botocore/client.py:1023 in _make_api_call                │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1024 │   │   else:                                                                             │
│   1025 │   │   │   return parsed_response                                                        │
│   1026                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) 
from eoPMC8WlwkK9Vy2ej1VetkERNMpOCdfUnLa8 with 

In [48]:
# from sagemaker import ModelPackage
# from sagemaker import get_execution_role
# from sagemaker.huggingface import HuggingFaceModel

# # Get the latest version of the model registered
# model_package = ModelPackage(
#     model_package_arn="arn:aws:sagemaker:eu-central-1:643202173500:model-package/huggingface-pytorch-training/1"
# )

# # Step 2: Create a SageMaker Model from the registered model
# model = HuggingFaceModel(
#     model_data=model_package.model_data,
#     role=role,
#     image_uri=model_package.image_uri,
#     transformers_version="4.28.1",                             # Transformers version used
#     pytorch_version="2.0.0",                                  # PyTorch version used
#     py_version='py310', 
#     env={
#         'HF_TASK': 'fill-mask'  # Set the task type here
#     }
    
# )

# # Step 3: Deploy the model to an endpoint
# predictor = model.deploy(
#     initial_instance_count=1,
#     instance_type='ml.m5.large'
# )



[11/20/24 21:55:36] INFO     Creating model with name:                                              ]8;id=764345;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=343674;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4025\4025]8;;\
                             huggingface-pytorch-inference-2024-11-20-21-55-36-066                                 

                    INFO     Creating endpoint-config with name                                     ]8;id=226504;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=908461;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#5820\5820]8;;\
                             huggingface-pytorch-inference-2024-11-20-21-55-36-727                                 

[11/20/24 21:55:37] INFO     Creating endpoint with name                                            ]8;id=306276;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=608335;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4642\4642]8;;\
                             huggingface-pytorch-inference-2024-11-20-21-55-36-727                                 

-------------!

In [60]:
print(model_package.model_data)

None


In [49]:
# Step 4: Make a prediction (example for text input)
response = predictor.predict({
    'inputs': 'This is a test sentence for BERT model prediction.'
})

print(response)

# Step 5: Clean up the endpoint when done


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # Step 4: Make a prediction (example for text input)                                         │
│ ❱ 2 response = predictor.predict({                                                               │
│   3 │   'inputs': 'This is a test sentence for BERT model prediction.'                           │
│   4 })                                                                                           │
│   5                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/base_predictor.py:212 in predict               │
│                                                                                                  │
│   209 │   │   if inference_component_name:                                                       │
│   210 │   │   │   request_args["InferenceComponentName"] = inference_component_name              │
│   211 │   │                                                                                      │
│ ❱ 212 │   │   response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**req   │
│   213 │   │   return self._handle_response(response)                                             │
│   214 │                                                                                          │
│   215 │   def _handle_response(self, response):                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/botocore/client.py:1023 in _make_api_call                │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1024 │   │   else:                                                                             │
│   1025 │   │   │   return parsed_response                                                        │
│   1026                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModelError: An error occurred (ModelError) when calling the I

In [84]:
predictor.delete_endpoint()

[11/20/24 23:03:55] INFO     Deleting endpoint configuration with name:                             ]8;id=214222;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=801180;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4796\4796]8;;\
                             huggingface-pytorch-inference-2024-11-2-2024-11-20-22-35-41-385                       

                    INFO     Deleting endpoint with name:                                           ]8;id=260271;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=644455;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4786\4786]8;;\
                             huggingface-pytorch-inference-2024-11-2-2024-11-20-22-35-41-385                       

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 predictor.delete_endpoint()                                                                  │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/base_predictor.py:529 in delete_endpoint       │
│                                                                                                  │
│   526 │   │   if delete_endpoint_config:                                                         │
│   527 │   │   │   self._delete_endpoint_config()                                                 │
│   528 │   │                                                                                      │
│ ❱ 529 │   │   self.sagemaker_session.delete_endpoint(self.endpoint_name)                         │
│   530 │                                                                                          │
│   531 │   def delete_predictor(self, wait: bool = False) -> None:                                │
│   532 │   │   """Delete the Amazon SageMaker inference component or endpoint backing this pred   │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/session.py:4787 in delete_endpoint             │
│                                                                                                  │
│   4784 │   │   │   endpoint_name (str): Name of the Amazon SageMaker ``Endpoint`` to delete.     │
│   4785 │   │   """                                                                               │
│   4786 │   │   logger.info("Deleting endpoint with name: %s", endpoint_name)                     │
│ ❱ 4787 │   │   self.sagemaker_client.delete_endpoint(EndpointName=endpoint_name)                 │
│   4788 │                                                                                         │
│   4789 │   def delete_endpoint_config(self, endpoint_config_name):                               │
│   4790 │   │   """Delete an Amazon SageMaker endpoint configuration.                             │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/botocore/client.py:1023 in _make_api_call                │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                       

In [ ]:
from sagemaker.workflow.pipeline_context import PipelineSession
pipeline_session = PipelineSession()

huggingface_estimator = HuggingFace(entry_point='train.py',
                            source_dir='./scripts',
                            base_job_name='huggingface-sdk-extension',
                            instance_type='ml.p3.2xlarge',
                            instance_count=1,
                            transformers_version="4.28.1",                             # Transformers version used
                            pytorch_version="2.0.0",                                  # PyTorch version used
                            py_version='py310', 
                            role=role,
                            sagemaker_session  = pipeline_session,
                            hyperparameters = {'epochs': 1,
                                               'train_batch_size': 32,
                                               'model_name':'distilbert-base-uncased'
                                                })